In [ ]:
import os, re, gc, json, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from datetime import timedelta
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from tqdm import tqdm

# =========================
# 설정 및 경로
# =========================
SEED = 42
np.random.seed(SEED)

# 🔧 경로 설정
BASE_PATH = "/content/drive/MyDrive/aimers"
TRAIN_PATH = f"{BASE_PATH}/train/train.csv"
TEST_DIR = f"{BASE_PATH}/test"
SAMPLE_SUB_PATH = f"{BASE_PATH}/sample_submission.csv"
OUTPUT_PATH = f"{BASE_PATH}/submission_test.csv"

# 연회장 처리 모드: "together" (연회장 포함 처리) | "separate" (연회장 따로 처리)
BANQUET_MODE = "separate"

# 저장 파일 경로 (연회장 탐지 설정)
BANQUET_DICT_PATH = f"{BASE_PATH}/banquet_detection_dictionary.json"
BANQUET_THRESH_PATH = f"{BASE_PATH}/banquet_thresholds.json"

print(f"🔧 Base Path: {BASE_PATH}")
print(f"🏢 Banquet Mode: {BANQUET_MODE}")

# =========================
# 공휴일 처리 (폴백 포함)
# =========================
try:
    import holidays
    _HAS_HOLIDAYS = True
    print("📅 Holidays 라이브러리 사용 가능")
except Exception:
    _HAS_HOLIDAYS = False
    print("⚠️ Holidays 라이브러리 없음, 기본 공휴일 사용")

def _is_holiday_date(d):
    if _HAS_HOLIDAYS:
        KR_HOLIDAYS = holidays.KR(years=[2023, 2024, 2025])
        return d in KR_HOLIDAYS
    BASIC_HOLI = {(m, d) for m, d in [(1,1),(3,1),(5,5),(6,6),(8,15),(10,3),(10,9),(12,25)]}
    return (d.month, d.day) in BASIC_HOLI

# =========================
# 평가 메트릭 (0 제외 SMAPE)
# =========================
def smape_ignore_zero(y_true, y_pred):
    mask = (y_true != 0)
    if mask.sum() == 0:
        return 0.0
    yt = y_true[mask]
    yp = y_pred[mask]
    denom = (np.abs(yt) + np.abs(yp))
    denom[denom == 0] = 1e-9
    return np.mean(2.0 * np.abs(yt - yp) / denom)

def lgb_smape_eval(y_pred, dataset):
    y_true = dataset.get_label()
    return 'smape_no0', smape_ignore_zero(y_true, y_pred), False

# =========================
# 유틸리티 함수
# =========================
def split_store_menu(x):
    """영업장명_메뉴명을 영업장명과 메뉴명으로 분리"""
    parts = str(x).split('_', 1)
    if len(parts) == 1:
        return parts[0], ''
    return parts[0], parts[1]

def ensure_daily_continuity(df):
    """결측 날짜를 0으로 채워 일별 연속성 보장"""
    out = []
    for k, g in df.groupby('영업장명_메뉴명', sort=False):
        g = g.sort_values('영업일자').copy()
        full = pd.DataFrame({
            '영업일자': pd.date_range(g['영업일자'].min(), g['영업일자'].max(), freq='D')
        })
        full['영업장명_메뉴명'] = k
        full = full.merge(g, on=['영업일자','영업장명_메뉴명'], how='left')
        full['매출수량'] = full['매출수량'].fillna(0.0)
        out.append(full)
    return pd.concat(out, ignore_index=True)

# =========================
# 피처 엔지니어링
# =========================
PEAK_MONTHS = {1,2,7,8,12}

def add_calendar_features(df):
    """달력 관련 피처 추가"""
    df['영업일자'] = pd.to_datetime(df['영업일자'])
    df['year'] = df['영업일자'].dt.year
    df['month'] = df['영업일자'].dt.month
    df['day'] = df['영업일자'].dt.day
    df['dow'] = df['영업일자'].dt.weekday  # 0=월요일, 6=일요일
    df['week'] = df['영업일자'].dt.isocalendar().week.astype(int)
    df['is_weekend'] = (df['dow'] >= 5).astype(int)
    df['is_holiday'] = df['영업일자'].dt.date.map(lambda d: int(_is_holiday_date(d)))
    df['is_pre_holiday'] = df['영업일자'].dt.date.map(lambda d: int(_is_holiday_date(d + timedelta(days=1))))
    df['is_post_holiday'] = df['영업일자'].dt.date.map(lambda d: int(_is_holiday_date(d - timedelta(days=1))))
    df['is_peak'] = df['month'].isin(PEAK_MONTHS).astype(int)
    df['month_dow'] = df['month'].astype(str) + '_' + df['dow'].astype(str)
    return df

def add_ts_features(df, lags=(1,2,3,5,7,14,21,28), roll_windows=(3,7,14,21,28,56)):
    """시계열 라그 및 롤링 피처 추가"""
    df = df.sort_values(['영업장명_메뉴명','영업일자']).copy()
    g_series = df.groupby('영업장명_메뉴명')['매출수량']

    # 라그 피처
    for L in lags:
        df[f'lag_{L}'] = g_series.shift(L)

    # 롤링 통계 피처
    for W in roll_windows:
        df[f'rmean_{W}'] = g_series.shift(1).rolling(W, min_periods=1).mean()
        df[f'rmedian_{W}'] = g_series.shift(1).rolling(W, min_periods=1).median()
        df[f'rstd_{W}'] = g_series.shift(1).rolling(W, min_periods=1).std()

    # 요일별 평균
    df['_prev'] = g_series.shift(1)
    df['dow_mean_14'] = (
        df.groupby(['영업장명_메뉴명','dow'])['_prev']
          .transform(lambda s: s.rolling(14, min_periods=1).mean())
    )
    df.drop(columns=['_prev'], inplace=True)
    return df

# =========================
# 라벨 인코딩
# =========================
def fit_label_encoders(df):
    """라벨 인코더 학습"""
    store, menu = zip(*df['영업장명_메뉴명'].map(split_store_menu))
    df['_store'] = store
    df['_menu'] = menu

    le_store = LabelEncoder().fit(df['_store'])
    le_menu = LabelEncoder().fit(df['_menu'])
    le_month_dow = LabelEncoder().fit(df['month_dow'])

    return le_store, le_menu, le_month_dow

def apply_label_encoders(df, le_store, le_menu, le_month_dow):
    """라벨 인코딩 적용"""
    store, menu = zip(*df['영업장명_메뉴명'].map(split_store_menu))
    df['_store'] = store
    df['_menu'] = menu

    df['_store_le'] = df['_store'].map(lambda x: le_store.transform([x])[0] if x in le_store.classes_ else -1)
    df['_menu_le'] = df['_menu'].map(lambda x: le_menu.transform([x])[0] if x in le_menu.classes_ else -1)
    df['_month_dow_le'] = df['month_dow'].map(lambda x: le_month_dow.transform([x])[0] if x in le_month_dow.classes_ else -1)

    return df

# =========================
# 연회장 탐지 시스템
# =========================
_DEFAULT_BANQUET_DICT = {
    "strong_regex_patterns": [
        r"(대|소)?\s*연회(장)?",
        r"연\s*회\s*장",
        r"웨딩(\s*홀)?",
        r"컨벤션(\s*홀)?",
        r"(그랜드)?\s*볼룸|ballroom",
        r"banquet",
        r"function\s*room",
        r"대관",
        r"예식",
        r"행사(\s*장)?",
    ],
    "whitelist_tokens": [
        "연회","연회장","대연회","소연회","대연회장","소연회장",
        "웨딩","웨딩홀","컨벤션","컨벤션홀","볼룸","그랜드볼룸",
        "banquet","ballroom","function","대관","예식","행사","연회코스","연회세트"
    ],
    "blacklist_tokens": [
        "홀","룸","룸1","룸2","룸A","룸B","룸C","홀A","홀B","홀C","룸(소)","룸(대)"
    ],
    "notes": "정규식 우선 적용. 단일 위험 토큰은 안전 컨텍스트에서만 허용"
}

def load_banquet_dict(path=None):
    """연회장 탐지 사전 로드"""
    if path and os.path.exists(path):
        try:
            with open(path, "r", encoding="utf-8") as f:
                return json.load(f)
        except Exception as e:
            print(f"⚠️ 연회장 사전 로드 실패: {e}")
    return _DEFAULT_BANQUET_DICT

def banquet_name_match(store, menu, dict_):
    """이름 기반 연회장 매칭"""
    s = str(store).lower()
    m = str(menu).lower()
    full_texts = [s, m, f"{s}_{m}"]

    # 강한 정규식 패턴 체크
    for pattern in dict_["strong_regex_patterns"]:
        regex = re.compile(pattern, re.IGNORECASE)
        if any(regex.search(text or "") for text in full_texts):
            return True

    # 토큰 기반 체크
    def tokenize(text):
        return [t for t in re.split(r"[^0-9A-Za-z가-힣]+", str(text)) if t]

    tokens = set(tokenize(s) + tokenize(m))

    # 화이트리스트 체크
    if any(tok.lower() in [w.lower() for w in dict_["whitelist_tokens"]] for tok in tokens):
        return True

    # 블랙리스트만 있으면 False
    if any(tok in dict_["blacklist_tokens"] for tok in tokens):
        return False

    return False

# 연회장 임계값 (기본값)
DEFAULT_BANQUET_THRESHOLDS = {
    "var": 2.16,
    "weekend": 4.92,
    "holiday": 4.92
}

def save_banquet_thresholds(thresholds, path):
    """연회장 임계값 저장"""
    try:
        with open(path, "w", encoding="utf-8") as f:
            json.dump(thresholds, f, ensure_ascii=False, indent=2)
    except Exception as e:
        print(f"⚠️ 임계값 저장 실패: {e}")

def load_banquet_thresholds(path):
    """연회장 임계값 로드"""
    if os.path.exists(path):
        try:
            with open(path, "r", encoding="utf-8") as f:
                return json.load(f)
        except Exception as e:
            print(f"⚠️ 임계값 로드 실패: {e}")
    return None

def fit_banquet_thresholds_from_train(train_df):
    """학습 데이터에서 연회장 통계 임계값 학습"""
    df = train_df.copy()
    df['영업일자'] = pd.to_datetime(df['영업일자'])
    df.loc[df['매출수량'] < 0, '매출수량'] = 0.0
    df['dow'] = df['영업일자'].dt.weekday
    df['is_weekend'] = (df['dow'] >= 5).astype(int)
    df['is_holiday'] = df['영업일자'].dt.date.map(lambda d: int(_is_holiday_date(d)))

    records = []
    for key, g in df.groupby("영업장명_메뉴명", sort=False):
        g = g.sort_values("영업일자")
        if len(g) == 0:
            continue

        # 최근 28일 데이터 추출
        last_day = g['영업일자'].max()
        recent = g[g['영업일자'] >= (last_day - pd.Timedelta(days=27))]
        if len(recent) < 14:
            recent = g.tail(28)

        sales = recent['매출수량'].astype(float).values
        if len(sales) == 0:
            continue

        # 변동성 비율
        mean_all = np.nanmean(sales)
        std_all = np.nanstd(sales)
        var_ratio = std_all / (mean_all + 1e-6)

        # 주말 비율
        weekday_sales = recent[recent['is_weekend'] == 0]['매출수량'].values
        weekend_sales = recent[recent['is_weekend'] == 1]['매출수량'].values
        mean_weekday = np.nanmean(weekday_sales) if len(weekday_sales) > 0 else np.nan
        mean_weekend = np.nanmean(weekend_sales) if len(weekend_sales) > 0 else np.nan
        weekend_ratio = (mean_weekend / (mean_weekday + 1e-6)) if not np.isnan(mean_weekday) and not np.isnan(mean_weekend) else np.nan

        # 공휴일 비율
        holiday_sales = recent[recent['is_holiday'] == 1]['매출수량'].values
        non_holiday_sales = recent[recent['is_holiday'] == 0]['매출수량'].values
        mean_holiday = np.nanmean(holiday_sales) if len(holiday_sales) > 0 else np.nan
        mean_non_holiday = np.nanmean(non_holiday_sales) if len(non_holiday_sales) > 0 else np.nan
        holiday_ratio = (mean_holiday / (mean_non_holiday + 1e-6)) if not np.isnan(mean_holiday) and not np.isnan(mean_non_holiday) else np.nan

        records.append((key, var_ratio, weekend_ratio, holiday_ratio))

    stat_df = pd.DataFrame(records, columns=["name","var_ratio","weekend_ratio","holiday_ratio"])
    clean_df = stat_df.replace([np.inf, -np.inf], np.nan).dropna()

    if len(clean_df) < 20:
        return DEFAULT_BANQUET_THRESHOLDS

    thresholds = {
        "var": float(clean_df['var_ratio'].quantile(0.90)),
        "weekend": float(clean_df['weekend_ratio'].quantile(0.90)),
        "holiday": float(clean_df['holiday_ratio'].quantile(0.90)),
    }
    return thresholds

def banquet_stats_match(group_df, thresholds):
    """통계 기반 연회장 매칭"""
    g = group_df.sort_values('영업일자')
    if len(g) < 14:
        return False

    # 최근 데이터 추출
    last_day = g['영업일자'].max()
    recent = g[g['영업일자'] >= (last_day - pd.Timedelta(days=27))].copy()
    if len(recent) < 14:
        recent = g.tail(28).copy()

    sales = recent['매출수량'].values.astype(float)
    mean_all = np.nanmean(sales)
    std_all = np.nanstd(sales)
    var_ratio = std_all / (mean_all + 1e-6)
    cond_var = (var_ratio > thresholds.get("var", 2.16))

    # 주말 조건
    cond_weekend = False
    if 'is_weekend' in recent.columns:
        weekday_sales = recent[recent['is_weekend'] == 0]['매출수량'].values
        weekend_sales = recent[recent['is_weekend'] == 1]['매출수량'].values
        if len(weekday_sales) > 0 and len(weekend_sales) > 0:
            mean_weekday = np.nanmean(weekday_sales)
            mean_weekend = np.nanmean(weekend_sales)
            if not np.isnan(mean_weekday) and not np.isnan(mean_weekend):
                weekend_ratio = mean_weekend / max(mean_weekday, 1e-6)
                cond_weekend = weekend_ratio > thresholds.get("weekend", 4.92)

    # 공휴일 조건
    cond_holiday = False
    if 'is_holiday' in recent.columns:
        holiday_sales = recent[recent['is_holiday'] == 1]['매출수량'].values
        non_holiday_sales = recent[recent['is_holiday'] == 0]['매출수량'].values
        if len(holiday_sales) > 0 and len(non_holiday_sales) > 0:
            mean_holiday = np.nanmean(holiday_sales)
            mean_non_holiday = np.nanmean(non_holiday_sales)
            if not np.isnan(mean_holiday) and not np.isnan(mean_non_holiday):
                holiday_ratio = mean_holiday / max(mean_non_holiday, 1e-6)
                cond_holiday = holiday_ratio > thresholds.get("holiday", 4.92)

    # 3개 조건 중 2개 이상 만족
    return (int(cond_var) + int(cond_weekend) + int(cond_holiday)) >= 2

def decide_banquet_for_group(key, group_df, banquet_dict, thresholds):
    """그룹별 연회장 여부 결정 (이름 + 통계 하이브리드)"""
    store, menu = split_store_menu(key)

    # 이름 기반 매칭이 우선
    name_match = banquet_name_match(store, menu, banquet_dict)
    if name_match:
        return 1

    # 통계 기반 매칭
    stats_match = banquet_stats_match(group_df, thresholds)
    return int(stats_match)

def add_banquet_flag(df, banquet_dict, thresholds):
    """연회장 플래그 추가"""
    flags = []

    # 캘린더 피처가 없으면 추가
    if 'dow' not in df.columns or 'is_weekend' not in df.columns or 'is_holiday' not in df.columns:
        df_tmp = add_calendar_features(df.copy())
    else:
        df_tmp = df.copy()

    for key, group in df_tmp.groupby('영업장명_메뉴명', sort=False):
        flag = decide_banquet_for_group(key, group, banquet_dict, thresholds)
        flags.append((key, flag))

    flag_df = pd.DataFrame(flags, columns=['영업장명_메뉴명', 'is_banquet'])
    return df.merge(flag_df, on='영업장명_메뉴명', how='left')

# =========================
# 모델 학습
# =========================
def train_ensemble_models(X_train, y_train, X_val, y_val):
    """앙상블 모델 학습"""
    models = []

    # 모델 1
    params1 = {
        'objective': 'poisson',
        'metric': 'None',
        'learning_rate': 0.025,
        'num_leaves': 640,
        'max_depth': -1,
        'min_data_in_leaf': 80,
        'feature_fraction': 0.90,
        'bagging_fraction': 0.90,
        'bagging_freq': 1,
        'lambda_l1': 0.0,
        'lambda_l2': 0.1,
        'verbose': -1,
        'seed': SEED,
        'force_row_wise': True
    }

    model1 = lgb.train(
        params1,
        lgb.Dataset(X_train, label=y_train),
        num_boost_round=9000,
        valid_sets=[lgb.Dataset(X_val, label=y_val)],
        feval=lgb_smape_eval,
        callbacks=[lgb.early_stopping(stopping_rounds=400, verbose=False)]
    )
    models.append(model1)

    # 모델 2
    params2 = {
        'objective': 'poisson',
        'metric': 'None',
        'learning_rate': 0.03,
        'num_leaves': 512,
        'max_depth': -1,
        'min_data_in_leaf': 100,
        'feature_fraction': 0.85,
        'bagging_fraction': 0.90,
        'bagging_freq': 1,
        'lambda_l1': 0.1,
        'lambda_l2': 0.1,
        'verbose': -1,
        'seed': SEED + 1,
        'force_row_wise': True
    }

    model2 = lgb.train(
        params2,
        lgb.Dataset(X_train, label=y_train),
        num_boost_round=9000,
        valid_sets=[lgb.Dataset(X_val, label=y_val)],
        feval=lgb_smape_eval,
        callbacks=[lgb.early_stopping(stopping_rounds=300, verbose=False)]
    )
    models.append(model2)

    return models

# =========================
# 예측 함수
# =========================
def prepare_features_block(df_block, le_store, le_menu, le_month_dow, banquet_dict, thresholds):
    """예측용 피처 블록 준비"""
    df_block = add_calendar_features(df_block)
    df_block = add_banquet_flag(df_block, banquet_dict, thresholds)
    df_block = add_ts_features(df_block)
    df_block = apply_label_encoders(df_block, le_store, le_menu, le_month_dow)
    return df_block

def one_step_predict(series_df, predict_date, model, features, le_store, le_menu, le_month_dow, banquet_dict, thresholds):
    """1일 앞 예측"""
    tmp = series_df.copy()
    new_row = pd.DataFrame({
        '영업일자': [predict_date],
        '영업장명_메뉴명': [series_df['영업장명_메뉴명'].iloc[0]],
        '매출수량': [np.nan]
    })
    tmp = pd.concat([tmp, new_row], ignore_index=True)
    tmp = prepare_features_block(tmp, le_store, le_menu, le_month_dow, banquet_dict, thresholds)

    row = tmp[tmp['영업일자'] == predict_date].iloc[-1]
    x = row[features].values.reshape(1, -1)
    pred = model.predict(x, num_iteration=getattr(model, "best_iteration", None))[0]
    return max(0.0, float(pred))

def forecast_7days_for_file(test_df, model_selector, features, le_store, le_menu, le_month_dow, banquet_dict, thresholds, test_prefix):
    """테스트 파일에 대한 7일 예측"""
    test_df = test_df.copy()
    test_df['영업일자'] = pd.to_datetime(test_df['영업일자'])
    test_df.loc[test_df['매출수량'] < 0, '매출수량'] = 0.0

    predictions = []

    for key, group in tqdm(test_df.groupby('영업장명_메뉴명', sort=False), desc=f"Forecasting {test_prefix}", leave=False):
        group = group.sort_values('영업일자').copy()

        # 연속성 보장
        full_range = pd.DataFrame({
            '영업일자': pd.date_range(group['영업일자'].min(), group['영업일자'].max(), freq='D')
        })
        full_range['영업장명_메뉴명'] = key
        full_range = full_range.merge(group, on=['영업일자','영업장명_메뉴명'], how='left')
        full_range['매출수량'] = full_range['매출수량'].fillna(0.0)

        # 연회장 여부 판단 및 모델 선택
        banquet_flag = decide_banquet_for_group(key, add_calendar_features(full_range.copy()), banquet_dict, thresholds)

        if isinstance(model_selector, dict):  # separate 모드
            models = model_selector['banquet'] if banquet_flag else model_selector['non']
        else:  # together 모드
            models = model_selector

        # 7일 예측
        last_day = full_range['영업일자'].max()
        current_data = full_range[['영업일자','영업장명_메뉴명','매출수량']].copy()

        daily_predictions = []
        for horizon in range(1, 8):
            prediction_date = last_day + pd.Timedelta(days=horizon)

            # 앙상블 예측
            ensemble_pred = np.mean([
                one_step_predict(current_data, prediction_date, model, features, le_store, le_menu, le_month_dow, banquet_dict, thresholds)
                for model in models
            ])

            daily_predictions.append(ensemble_pred)

            # 예측 결과를 다음 예측을 위해 추가
            new_row = pd.DataFrame({
                '영업일자': [prediction_date],
                '영업장명_메뉴명': [key],
                '매출수량': [ensemble_pred]
            })
            current_data = pd.concat([current_data, new_row], ignore_index=True)

        # 결과 저장
        for horizon, pred in enumerate(daily_predictions, 1):
            predictions.append([key, test_prefix, horizon, pred])

    result_df = pd.DataFrame(predictions, columns=['영업장명_메뉴명','test_prefix','offset','매출수량'])
    return result_df
    # =========================
# 메인 실행 부분
# =========================
# 학습 진행률을 보여주는 LightGBM용 콜백 함수
def create_lgbm_tqdm_callback(num_boost_round):
    """Create a tqdm callback for LightGBM training."""
    pbar = tqdm(total=num_boost_round, desc="🚀 Training", leave=False)

    def callback(env):
        current_iter = env.iteration
        best_iter = getattr(env, "best_iteration", -1)

        # update the progress bar
        pbar.update(1)
        pbar.set_postfix({
            'Current Iter': current_iter,
            'Best Iter': best_iter if best_iter != -1 else 'N/A'
        })

        # check for early stopping and close pbar
        if env.early_stopping_round is not None and current_iter >= env.best_iteration:
            pbar.close()

    return callback

if __name__ == '__main__':
    print("🚀 연회장 분리 처리 시스템 시작")

    # 1. 연회장 탐지 설정 로드
    banquet_dict = load_banquet_dict(BANQUET_DICT_PATH)
    print(f"📖 연회장 사전 로드 완료 (패턴 {len(banquet_dict['strong_regex_patterns'])}개)")

    # 2. 학습 데이터 로드 및 전처리
    print("📂 학습 데이터 로딩 중...")
    train = pd.read_csv(TRAIN_PATH)
    train['영업일자'] = pd.to_datetime(train['영업일자'])
    train.loc[train['매출수량'] < 0, '매출수량'] = 0.0
    print(f"✅ 학습 데이터 로드 완료: {len(train):,} 행")

    # 3. 연회장 임계값 설정
    loaded_thresholds = load_banquet_thresholds(BANQUET_THRESH_PATH)
    if loaded_thresholds is None:
        print("🔍 학습 데이터에서 연회장 임계값 추정 중...")
        banquet_thresholds = fit_banquet_thresholds_from_train(train)
        save_banquet_thresholds(banquet_thresholds, BANQUET_THRESH_PATH)
        print(f"✅ 임계값 추정 완료 - Var: {banquet_thresholds['var']:.3f}, Weekend: {banquet_thresholds['weekend']:.3f}, Holiday: {banquet_thresholds['holiday']:.3f}")
    else:
        banquet_thresholds = loaded_thresholds
        print(f"✅ 기존 연회장 임계값 로드 완료")

    # 4. 피처 엔지니어링 및 데이터 분할
    print("🛠️ 피처 엔지니어링 및 데이터 분할 중...")

    # 학습 데이터에 일별 연속성 보장 및 피처 추가
    train_full = ensure_daily_continuity(train)
    train_full = add_calendar_features(train_full)
    train_full = add_banquet_flag(train_full, banquet_dict, banquet_thresholds)
    train_full = add_ts_features(train_full)

    # 라벨 인코더 학습
    le_store, le_menu, le_month_dow = fit_label_encoders(train_full)
    train_full = apply_label_encoders(train_full, le_store, le_menu, le_month_dow)

    # 학습 및 검증 데이터 분할
    TRAIN_CUTOFF = pd.to_datetime('2024-03-31')
    train_df = train_full[train_full['영업일자'] <= TRAIN_CUTOFF]
    val_df = train_full[train_full['영업일자'] > TRAIN_CUTOFF].copy()

    # 피처 및 타겟 설정
    TARGET = '매출수량'
    FEATURES = [
        'year','month','day','dow','week',
        'is_weekend','is_holiday','is_pre_holiday','is_post_holiday',
        'is_peak', 'is_banquet',
        '_store_le','_menu_le','_month_dow_le',
    ]
    FEATURES.extend([f'lag_{L}' for L in (1,2,3,5,7,14,21,28)])
    FEATURES.extend([f'rmean_{W}' for W in (3,7,14,21,28,56)])
    FEATURES.extend([f'rmedian_{W}' for W in (3,7,14,21,28,56)])
    FEATURES.extend([f'rstd_{W}' for W in (3,7,14,21,28,56)])
    FEATURES.append('dow_mean_14')

    # 누락된 데이터 제거
    X_train = train_df.dropna(subset=FEATURES).reset_index(drop=True)[FEATURES]
    y_train = train_df.dropna(subset=FEATURES).reset_index(drop=True)[TARGET]
    X_val = val_df[FEATURES]
    y_val = val_df[TARGET]

    print("✅ 데이터 전처리 완료")
    print(f"   학습 데이터: {len(X_train):,}개, 검증 데이터: {len(X_val):,}개")

    # 5. 모델 학습 (연회장 분리 모드)
    models = {}
    if BANQUET_MODE == "separate":
        print("🤖 연회장 데이터를 분리하여 모델 학습 시작...")

        # 연회장 그룹 분리
        banquet_train_df = train_df[train_df['is_banquet'] == 1].dropna(subset=FEATURES).reset_index(drop=True)
        non_banquet_train_df = train_df[train_df['is_banquet'] == 0].dropna(subset=FEATURES).reset_index(drop=True)

        banquet_val_df = val_df[val_df['is_banquet'] == 1]
        non_banquet_val_df = val_df[val_df['is_banquet'] == 0]

        # 연회장 모델 학습
        print(f"   - 연회장 모델 학습 중... ({len(banquet_train_df):,}개 데이터)")
        if len(banquet_train_df) > 100:
            models['banquet'] = train_ensemble_models(
                banquet_train_df[FEATURES], banquet_train_df[TARGET],
                banquet_val_df[FEATURES], banquet_val_df[TARGET]
            )
        else:
            print("     ⚠️ 연회장 데이터 부족, 전체 모델로 통합 학습합니다.")
            models['banquet'] = train_ensemble_models(X_train, y_train, X_val, y_val)

        # 일반 모델 학습
        print(f"   - 일반 모델 학습 중... ({len(non_banquet_train_df):,}개 데이터)")
        models['non'] = train_ensemble_models(
            non_banquet_train_df[FEATURES], non_banquet_train_df[TARGET],
            non_banquet_val_df[FEATURES], non_banquet_val_df[TARGET]
        )
        print("✅ 모델 학습 완료")

    else: # together 모드
        print("🤖 전체 데이터를 통합하여 모델 학습 시작...")
        models['all'] = train_ensemble_models(X_train, y_train, X_val, y_val)
        print("✅ 모델 학습 완료")

    # 6. 테스트 데이터 로드 및 예측
    print("📋 테스트 데이터 로딩 및 예측 시작...")
    submission = pd.read_csv(SAMPLE_SUB_PATH)
    test_files = [f for f in os.listdir(TEST_DIR) if f.endswith('.csv')]

    all_predictions = []

    # 전체 테스트 파일에 대한 예측 진행
    for test_file in tqdm(test_files, desc="Processing Test Files"):
        test_prefix = test_file.replace('.csv', '')
        test_path = os.path.join(TEST_DIR, test_file)
        test_df = pd.read_csv(test_path)

        if BANQUET_MODE == "separate":
            test_models = {
                'banquet': models.get('banquet'),
                'non': models.get('non')
            }
        else:
            test_models = models.get('all')

        file_predictions = forecast_7days_for_file(
            test_df=test_df,
            model_selector=test_models,
            features=FEATURES,
            le_store=le_store,
            le_menu=le_menu,
            le_month_dow=le_month_dow,
            banquet_dict=banquet_dict,
            thresholds=banquet_thresholds,
            test_prefix=test_prefix
        )
        all_predictions.append(file_predictions)
        gc.collect() # 메모리 정리

    print("✅ 모든 테스트 파일 예측 완료")

# 7. 제출 파일 생성
final_pred_df = pd.concat(all_predictions, ignore_index=True)

# 💡 수정된 부분: 와이드 포맷의 제출 파일에 예측값 할당
print("✍️ 제출 파일 작성 중...")

# 제출 파일의 인덱스를 '영업일자'로 설정
submission.set_index('영업일자', inplace=True)

for _, row in tqdm(final_pred_df.iterrows(), total=len(final_pred_df), desc="Filling Submission"):
    store_menu = row['영업장명_메뉴명']
    test_prefix = row['test_prefix']
    offset = row['offset']
    prediction = row['매출수량']

    # 'TEST_01+1일', 'TEST_01+2일' 형식의 인덱스 이름 생성
    index_name = f"{test_prefix}+{offset}일"

    # 해당 위치에 예측값 할당
    if index_name in submission.index and store_menu in submission.columns:
        submission.loc[index_name, store_menu] = max(0, int(round(prediction)))

submission.reset_index(inplace=True)
submission.to_csv(OUTPUT_PATH, index=False)

print(f"🎉 제출 파일이 {OUTPUT_PATH}에 성공적으로 저장되었습니다.")